Steps we performed to setup database:

1. Download a csv file containing stock data for approximately 1000 companies from Quandl, for years 2010-2017: SHARADAR-SF1.csv
2. Download a csv file that describes the Quandl elements: SHARADAR-INDICATORS1.csv
3. Get ticker symbols from yahoo finance, to be able to get company names: Yahoo Ticker Symbols - September 2017.csv
4. Randomly pick 10 tickers/companies to model: A, AAPL, AMD, ARQL, AU, BHP, BLIN, BOSC, BP, CVX
5. Create a new schema/database called stockuser in MySQL
6. Import the Quandl stock data into MySQL, using the import wizard - approximately 10000 rows
7. Create a new table containing Yahoo Ticker symbols and company names.
8. Calculate stock returns for each year (((new stock price + dividends) - old stock price)/old stock price) * 100
9. Create a new table that contains only the rows/columns that we are concerned with.
10. Export a csv file of the table.
11. use scipy regression to generate regression values.

In order to get our project to work we will provide the cvs file needed from step 10. above.
below steps are required:

1. create user as below: stockuser
2. create table stocks10
3. insert rows into table from csv

In [1]:
# Dependencies
from matplotlib import pyplot as plt
from scipy import stats

import numpy as np
import pandas as pd

# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

#json
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)
import MySQLdb
import csv

In [11]:
#create user
db1 = MySQLdb.connect(host="localhost",user="root",passwd=<passwd>)
cursor = db1.cursor()
sql = 'CREATE DATABASE stockuser'
cursor.execute(sql)

#create table to hold stock data
sql = 'CREATE TABLE stockuser.stocks10 (company_name text, ticker text,calendardate text,\
pe text, pb text,fcf text,stock_return text) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci;'
cursor.execute(sql)

#inserting rows into table
with open('stocks_10_selected.csv', newline='') as myFile:  
    reader = csv.reader(myFile)
    for row in reader:
        print(row)
        cursor.execute("INSERT INTO stockuser.stocks10(company_name,ticker,calendardate,pe,pb,fcf,stock_return) VALUES(%s,%s,%s,%s,%s,%s,%s)", row)
#commit and close the connection
db1.commit()
cursor.close()


['company_name', 'ticker', 'calendardate', 'pe', 'pb', 'fcf', 'stock_return']
['Agilent Technologies, Inc.', 'A', '12/31/2012', '10.876', '2.42', '1034000000', '-2.913407068']
['Agilent Technologies, Inc.', 'A', '12/31/2013', '22.876', '3.177', '959000000', '41.83917755']
['Agilent Technologies, Inc.', 'A', '12/31/2014', '33.582', '3.478', '540000000', '9.80464933']
['Agilent Technologies, Inc.', 'A', '12/31/2015', '31.206', '3.003', '426000000', '-30.69319826']
['Agilent Technologies, Inc.', 'A', '12/31/2016', '30.592', '3.331', '654000000', '57.88665254']
['Agilent Technologies, Inc.', 'A', '12/31/2017', '32.009', '4.532', '713000000', '57.23954556']
['Apple Inc.', 'AAPL', '12/31/2012', '14.985', '5.29', '41454000000', '65.66338972']
['Apple Inc.', 'AAPL', '12/31/2013', '11.842', '3.55', '44590000000', '-25.49885101']
['Apple Inc.', 'AAPL', '12/31/2014', '15.269', '5.408', '49900000000', '51.67310481']
['Apple Inc.', 'AAPL', '12/31/2015', '12.252', '5.481', '69778000000', '19.6128139

In [12]:
# Create Engine and Pass in MySQL Connection
engine = create_engine("mysql://root:<passwd>@localhost:3306/stockuser")
conn = engine.connect()

# Query Single Record in the the Database
#data = engine.execute("SELECT stock_return,pe FROM stocks WHERE ticker = 'A' and stock_return != 0;")
# Query All Records in the the Database
ticker='A'
query_string = "SELECT stock_return,pe,pb,fcf FROM stocks10 WHERE ticker = '"+ticker+"' and stock_return != 0;"
query_string
data = pd.read_sql(query_string, conn)

data.head()


C:\Users\Owner\Anaconda3\envs\Pythondata1\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,stock_return,pe,pb,fcf
0,-2.913407068,10.876,2.42,1034000000
1,41.83917755,22.876,3.177,959000000
2,9.80464933,33.582,3.478,540000000
3,-30.69319826,31.206,3.003,426000000
4,57.88665254,30.592,3.331,654000000


In [13]:
stockreturn = []
pe = []
pb = []
fcf = []
for row in data.iterrows():
    stockreturn.append(row[1].stock_return)
    pe.append(row[1].pe)
    pb.append(row[1].pb)
    fcf.append(row[1].fcf)
    
stockreturnint = [float(i) for i in stockreturn]
peint = [float(i) for i in pe]
pbint = [float(i) for i in pb]
fcfint = [float(i) for i in fcf]

In [15]:
pe_slope, pe_int, pe_r_value, pe_p_value, pe_std_err = stats.linregress(peint,stockreturnint)
pb_slope, pb_int, pb_r_value, pb_p_value, pb_std_err = stats.linregress(pbint,stockreturnint)
fcf_slope, fcf_int, fcf_r_value, fcf_p_value, fcf_std_err = stats.linregress(fcfint,stockreturnint)

regPE = [pe_slope, pe_int, pe_r_value, pe_p_value, pe_std_err]
regPB = [pb_slope, pb_int, pb_r_value, pb_p_value, pb_std_err]
regFCF = [fcf_slope, fcf_int, fcf_r_value, fcf_p_value, fcf_std_err]